In [1]:
"""
   Copyright 2023 Raphaël Isvelin

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
"""

VIEWER_THEME = "light"  # light, dark
VIEWER_HEIGHT = 900
ENABLE_REPLAY = True

import cadquery as cq

from jupyter_cadquery import show as jcq_show
from jupyter_cadquery import (
    versions,
    PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer, get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
enable_replay(ENABLE_REPLAY)

from cq_warehouse.fastener import (
    polygon_diagonal,
    read_fastener_parameters_from_csv,
    Screw, SocketHeadCapScrew, HeatSetNut, PlainWasher,
    HexNutWithFlange, HexHeadScrew, PanHeadScrew, CounterSunkScrew
)
import cq_warehouse.extensions

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay
Note: To get rid of this warning, use 'enable_replay(False)'


In [2]:
cv = open_viewer("Assembly", anchor="right")  # sets default viewer

objects_to_hide = [
    "/Masks"
]

def hide_objects_matching_strings(strings, keep_edges=True, cv=cv):
    for state in cv.widget.states:
        should_hide = False
        for s in strings:
            if s in state:
                should_hide = True
                break
        if should_hide:
            cv.update_states({
                state: (0, 0 if keep_edges else 0),
            })
            
def show(obj, viewer="Assembly", anchor="right", hide_contains=objects_to_hide):
    cv = jcq_show(
        obj,
        viewer=viewer,
        anchor=anchor,
        #cad_width=1640,
        height=VIEWER_HEIGHT,
        theme=VIEWER_THEME,
        collapse=1,
        optimal_bb=True,
        render_edges=True,
        axes=True,
        axes0=True,
        grid=[True, True, True],
        #black_edges=True,
        reset_camera=False,
        #show_parent=True,
        #timeit=True,
        #js_debug=True
    )
    hide_objects_matching_strings(hide_contains, cv=cv)

def show_b(obj, viewer="Quick", hide_contains=objects_to_hide):
    cv2 = open_viewer(viewer, anchor="split-bottom")
    show(obj, viewer, anchor="split-bottom", hide_contains=hide_contains)

def show_x(obj, hide_contains=objects_to_hide):
    show(obj, viewer=None, hide_contains=hide_contains)

def show_part(part: Part, bottom=False):
    footprint_assembly = (
        cq.Assembly()
            .add(part.debug_objects.footprint.inside, name="Inside", color=cq.Color(1, 0, 1))
            .add(part.debug_objects.footprint.outside, name="Outside", color=cq.Color(0, 1, 1))
    )
    test_assembly = (
        cq.Assembly(None, name="test_assembly")
            .add(part.mask, name="Mask", color=cq.Color(0, 1, 0))
            .add(part.part, name="Part")
            .add(part.assembly_parts_to_cq_assembly(), name="Part assembly")
            .add(footprint_assembly, name="Footprint")
    )
    debug_assembly = cq.Assembly()
    debug_assembly.add(part.debug_objects.hole, name="Hole", color=cq.Color(1, 0, 0))
    for debug_obj_name in part.debug_objects.others.keys():
        obj = part.debug_objects.others[debug_obj_name]
        if not isinstance(obj, dict):
            debug_assembly.add(obj, name=debug_obj_name, color=cq.Color(0.5, 0.7, 0.5))
    test_assembly.add(debug_assembly, name="Debug")

    if bottom: show_b(test_assembly)
    else: show(test_assembly)

## Build test parts

In [3]:
import sys
sys.path.append("../src")

from cq_enclosure_builder import PartFactory as pf
from cq_enclosure_builder import Panel, PanelSize
from cq_enclosure_builder import Enclosure, Face
from cq_enclosure_builder.knobs_and_caps import KNOB_12_6_x_15_8

pf.set_default_types({
    "jack": '6.35mm PJ-612A',
    "usb_a": '3.0 vertical cltgxdd',
    "usb_c": 'ChengHaoRan E',
    "button": 'SPST PBS-24B-4',
    "encoder": 'EC11',
    "screen": 'HDMI 5 inch JRP5015',
    "potentiometer": 'WH148',
    "banana": '4mm',
    "barrel_plug": 'DC-022B',
    "rca": 'N1030',
})
pf.set_default_parameters({
    "enclosure_wall_thickness": 2,
    "pot_knob": KNOB_12_6_x_15_8
})

usb_c = pf.build_usb_c()
usb_c_v = pf.build_usb_c(orientation_vertical=True)
usb_a = pf.build_usb_a()
usb_a_v = pf.build_usb_a(orientation_vertical=True)
spst = pf.build_button()
encoder = pf.build_encoder() 
jack_35 = pf.build_jack(part_type='3.5mm XXX')
jack_635 = pf.build_jack()
screen = pf.build_screen()
pot = pf.build_potentiometer()
banana = pf.build_banana()
barrel_plug = pf.build_barrel_plug()
rca = pf.build_rca()


show_part(pot)

VALIDATING CLASS: UsbCChengHaoRanEPart
VALIDATING CLASS: UsbCChengHaoRanEPart
VALIDATING CLASS: UsbA30VerticalCltgxddPart
VALIDATING CLASS: UsbA30VerticalCltgxddPart
VALIDATING CLASS: ButtonSpstPbs24b4Part
VALIDATING CLASS: EncoderEc11Part
VALIDATING CLASS: Jack3_5mmXxxPart
VALIDATING CLASS: Jack6_35mmPj612aPart
TODO Need to move all workplanes so the hole's center is at 0,0
VALIDATING CLASS: Hdmi5InchJrp5015Part
VALIDATING CLASS: PotentiometerWh148Part
VALIDATING CLASS: Banana4mmPart
VALIDATING CLASS: BarrelPlugDc022bPart
VALIDATING CLASS: RcaN1030Part
100% ⋮————————————————————————————————————————————————————————————⋮ (12/12)  0.16s


## Panels etc.

In [4]:
import cadquery as cq
from cq_enclosure_builder import Face
from cq_enclosure_builder.part import Part

class PanelSize:
    def __init__(self, width, length, wall_thickness, total_thickness):
        self.width = width
        self.length = length
        self.wall_thickness = wall_thickness
        self.total_thickness = total_thickness

class Panel:
    def __init__(self,
                 face: Face,
                 top_view_width,
                 top_view_length,
                 wall_thickness,
                 color=None,
                 part_color=None,
                 alpha=1.0,
                 lid_size_error_margin=0.0  # if provided, panel will be smaller than mask
        ):
        self.face = face
        self.size = PanelSize(top_view_width, top_view_length, wall_thickness, wall_thickness)
        self._color = color
        if self._color == None:
            self._color = self.face.default_color
        self._part_color = part_color
        if self._part_color == None:
            self._part_color = self.face.default_part_color
        self.panel = None
        self.mask = self._rotate_to_face(
            cq.Workplane("front")
                .workplane()
                .box(top_view_width, top_view_length, wall_thickness, centered=(True, True, False))
        )
        self.debug_assemblies = {}
        self.debug_assemblies["hole"] = None
        self.debug_assemblies["footprint_in"] = None
        self.debug_assemblies["footprint_out"] = None
        self.debug_assemblies["other"] = None
        self.debug_assemblies["combined"] = cq.Assembly(None, name=self.face.label + " - Debug")
        self.lid_size_error_margin = lid_size_error_margin
        self._alpha = alpha
        self._parts_to_add = []

    def add(self, label: str, part: Part, rel_pos=None, abs_pos=None):
        print(self.face.label + ": adding part '" + label + "'")
        pos = None
        if rel_pos == None and abs_pos == None:
            raise ValueError("Either rel_pos or abs_pos must be set.")
        elif rel_pos == None:
            pos = (
                abs_pos[0] - self.size.width/2,
                abs_pos[1] - self.size.length/2
            )
        elif abs_pos==None:
            pos = rel_pos
        self._parts_to_add.append({
            "part": part,
            "label": label,
            "pos": pos
        })
        return self

    def assemble(self):
        wall = (
            cq.Workplane("front")
                .workplane()
                .box(self.size.width - self.lid_size_error_margin, self.size.length - self.lid_size_error_margin, self.size.wall_thickness,
                     centered=(True, True, False))
        )
        self.panel = cq.Assembly(None, name="Panel TOP")
        for part_to_add in self._parts_to_add:
            part_obj = part_to_add["part"]
            self._add_part_to_debug_assemblies(part_to_add)
            wall = wall.cut(
                part_obj.mask.translate([*part_to_add["pos"], 0])
            )
            if part_obj.assembly_parts != None:
                self.panel = self.panel.add(
                    self._translate_assembly_objects_and_rotate_to_face(part_obj.assembly_parts, [*part_to_add["pos"], 0]))
                    #self._rotate_assembly_to_face(part_obj.part_assembly.translate([*part_to_add["pos"], 0])))
            else:
                self.panel = self.panel.add(
                    self._rotate_to_face(
                        part_obj.part.translate([*part_to_add["pos"], 0])),
                    name=part_to_add["label"],
                    color=cq.Color(*self._part_color, 1.0)
                )
            if part_obj.size.thickness > self.size.total_thickness:
                self.size.total_thickness = part_obj.size.thickness
        self.panel = self.panel.add(self._rotate_to_face(wall), name="Wall", color=cq.Color(*self._color, self._alpha))
        self.debug_assemblies["combined"] = self._build_combined_debug_assembly()
        return self

    def _rotate_to_face(self, wp):
        if self.face == Face.TOP:
            wp = wp.mirror("XY")
        elif self.face == Face.BOTTOM:
            wp = wp.mirror("XZ")
        elif self.face == Face.BACK:
            wp = wp.rotate((0, 0, 0), (1, 0, 0), 90)
            wp = wp.mirror("YZ")
        elif self.face == Face.FRONT:
            wp = wp.rotate((0, 0, 0), (1, 0, 0), 90)
            wp = wp.mirror("XZ")
        elif self.face == Face.LEFT:
            wp = wp.rotate((0, 0, 0), (1, 0, 0), 90)
            wp = wp.rotate((0, 0, 0), (0, 0, 1), 90)
            wp = wp.mirror("XZ")
        elif self.face == Face.RIGHT:
            wp = wp.rotate((0, 0, 0), (1, 0, 0), 90)
            wp = wp.rotate((0, 0, 0), (0, 0, 1), -90)
            wp = wp.mirror("XZ")
        return wp

    def _translate_assembly_objects_and_rotate_to_face(self, assembly_parts, translation):
        assembly = cq.Assembly()
        for assembly_part in assembly_parts:
            part, loc, name, color = assembly_part.as_assembly_add_parameters()
            part = part.translate(translation)
            part = self._rotate_to_face(part)
            assembly.add(part, name=name, color=color)
        return assembly

    def _add_part_to_debug_assemblies(self, part_to_add):
        part = part_to_add["part"]
        part_pos = part_to_add["pos"]
        part_label: str = part_to_add["label"]
        if part.debug_objects.hole != None:
            if self.debug_assemblies["hole"] == None:
                self.debug_assemblies["hole"] = cq.Assembly()
            hole = self._rotate_to_face(part.debug_objects.hole.translate([*part_pos, 0]))
            self.debug_assemblies["hole"] = self.debug_assemblies["hole"].add(hole, name=part_label, color=cq.Color(1, 0, 0))
        if part.debug_objects.footprint.inside != None:
            if self.debug_assemblies["footprint_in"] == None:
                self.debug_assemblies["footprint_in"] = cq.Assembly()
            footprint_in = self._rotate_to_face(part.debug_objects.footprint.inside.translate([*part_pos, 0]))
            self.debug_assemblies["footprint_in"] = self.debug_assemblies["footprint_in"].add(footprint_in, name=part_label, color=cq.Color(1, 0, 1))
        if part.debug_objects.footprint.outside != None:
            if self.debug_assemblies["footprint_out"] == None:
                self.debug_assemblies["footprint_out"] = cq.Assembly()
            footprint_out = self._rotate_to_face(part.debug_objects.footprint.outside.translate([*part_pos, 0]))
            self.debug_assemblies["footprint_out"] = self.debug_assemblies["footprint_out"].add(footprint_out, name=part_label, color=cq.Color(0, 1, 1))
        other_debug_assembly = None
        for key in part.debug_objects.others.keys():
            if other_debug_assembly == None:
                other_debug_assembly = cq.Assembly(None)
                self.debug_assemblies["other"] = cq.Assembly()
            debug_part = self._rotate_to_face(part.debug_objects.others[key].translate([*part_pos, 0]))
            other_debug_assembly = other_debug_assembly.add(debug_part, name=key)
        if other_debug_assembly != None:
            if self.debug_assemblies["other"] == None:
                self.debug_assemblies["other"] = cq.Assembly()
            self.debug_assemblies["other"] = self.debug_assemblies["other"].add(other_debug_assembly, name=part_label, color=cq.Color(1, 1, 0))

    def _build_combined_debug_assembly(self):
        combined = self.debug_assemblies["combined"]
        combined = combined.add(self.mask, name="Mask", color=cq.Color(0, 1, 0, 0.5))
        if self.debug_assemblies["hole"] != None:
            combined = combined.add(self.debug_assemblies["hole"], name="Holes")
        if self.debug_assemblies["footprint_in"] != None:
            combined = combined.add(self.debug_assemblies["footprint_in"], name="Footprint in")
        if self.debug_assemblies["footprint_out"] != None:
            combined = combined.add(self.debug_assemblies["footprint_out"], name="Footprint out")
        if self.debug_assemblies["other"] != None:
            combined = combined.add(self.debug_assemblies["other"], name="Other")
        return combined


In [5]:
from cq_enclosure_builder.parts.common.screw_block import ScrewBlock
from cq_enclosure_builder.parts.common.screws_providers import DefaultFlatHeadScrewProvider, DefaultHeadSetScrewProvider

screw = ScrewBlock(screw_provider=DefaultFlatHeadScrewProvider,
                   counter_sunk_screw_provider=DefaultFlatHeadScrewProvider
        ).m3(6, is_counter_sunk=True, with_counter_sunk_block=True)

a = (
    cq.Assembly(None, name="Screw block")
        .add(screw["block"], name="Block", color=cq.Color(1, 0, 0))
        .add(screw["mask"], name="Block Mask", color=cq.Color(0, 1, 0))
        .add(screw["counter_sunk_block"], name="CS Block", color=cq.Color(0, 1, 1))
        .add(screw["counter_sunk_mask"], name="CS Block Mask", color=cq.Color(0, 1, 0))
)
show(a)

Building CS cat m3 - ref M3-0.5
100% ⋮————————————————————————————————————————————————————————————⋮ (4/4)  0.28s


In [109]:
from typing import List

from cq_enclosure_builder.parts.common.screw_block import ScrewBlock, TaperOptions
from cq_enclosure_builder.parts.common.screws_providers import DefaultFlatHeadScrewProvider, DefaultHeadSetScrewProvider

def explode(pos_array, walls_explosion_factor=2.0):
    return [x * walls_explosion_factor for x in pos_array]

class EnclosureSize:
    def __init__(self, inner_width, inner_length, inner_thickness, wall_thickness):
        self.inner_width = inner_width
        self.inner_length = inner_length
        self.inner_thickness = inner_thickness
        self.wall_thickness = wall_thickness

class Enclosure:
    def __init__(
        self,
        size: EnclosureSize,
        lid_on_faces: List[Face] = [Face.BOTTOM],
        lid_panel_size_error_margin = 0.8,  # meaning the lid is `margin` smaller than the hole on both width and length
        lid_screws_thickness_error_margin = 0.4,
        add_corner_lid_screws = True,
        no_fillet_top = False,
        no_fillet_bottom = False,
        
    ):
        super().__init__()

        inner_width = size.inner_width
        inner_length = size.inner_length
        inner_thickness = size.inner_thickness
        wall_thickness = size.wall_thickness
        self.size = size
        self.no_fillet_top = no_fillet_top
        self.no_fillet_bottom = no_fillet_bottom

        if lid_on_faces != [Face.BOTTOM]:
            # TODO: add support (upddate the correct face in panels_specs)
            # TODO nice-to-have: also add support for multiple lids (faces)
            raise ValueError("lid_on_faces: only value supported for now is [BOTTOM], got " + str(lid_on_faces))

        self.frame = (
            cq.Workplane("front")
                .box(inner_width - 4, inner_length - 4, inner_thickness - 4, centered=(True, True, False))
                #.faces("+Z")
                .shell(wall_thickness)
                #.translate([2, 2, 2])
        )
        self.panels_specs = [
            (Face.TOP,    (inner_width-4,  inner_length-4,    wall_thickness),  [0, 0, inner_thickness - wall_thickness],   0.9 ),
            (Face.BOTTOM, (inner_width-4,  inner_length-4,    wall_thickness),  [0, 0, -wall_thickness],    0.9 ),
            (Face.FRONT,  (inner_width-4,  inner_thickness-4, wall_thickness),  [0, -(inner_length/2), inner_thickness/2 - wall_thickness], 0.9 ),
            (Face.BACK,   (inner_width-4,  inner_thickness-4, wall_thickness),  [0, inner_length/2, inner_thickness/2 - wall_thickness],  0.9 ),
            (Face.LEFT,   (inner_length-4, inner_thickness-4, wall_thickness),  [-(inner_width/2), 0, inner_thickness/2 - wall_thickness], 0.9 ),
            (Face.RIGHT,  (inner_length-4, inner_thickness-4, wall_thickness),  [inner_width/2, 0, inner_thickness/2 - wall_thickness],  0.9 ),
        ]

        self.panels = {}
        self.screws_specs = []
        self.screws = []

        for info in self.panels_specs:
            lid_size_error_margin = 0 if info[0] not in lid_on_faces else lid_panel_size_error_margin
            self.panels[info[0]] = Panel(info[0], *info[1], alpha=info[3], lid_size_error_margin=lid_size_error_margin)

        if add_corner_lid_screws:
            self.add_corner_lid_screws(lid_screws_thickness_error_margin)

    def add_part_to_face(self, face: Face, part_label: str, part: Part, rel_pos=None, abs_pos=None):
        self.panels[face].add(part_label, part, rel_pos, abs_pos)
        return self

    def add_screw(
        self,
        screw_size_category: str = "m3",
        rel_pos = None,
        abs_pos = None,
        pos_error_margin = 0,
        taper: TaperOptions = TaperOptions.NO_TAPER,
        taper_rotation: float = 0.0,
        screw_provider = DefaultFlatHeadScrewProvider,
        counter_sunk_screw_provider = DefaultFlatHeadScrewProvider
    ):
        # TODO support lid != Face.BOTTOM

        pos = None
        if rel_pos == None and abs_pos == None:
            raise ValueError("Either rel_pos or abs_pos must be set.")
        elif rel_pos == None:
            pos = (
                abs_pos[0] - self.size.inner_width/2,
                abs_pos[1] - self.size.inner_length/2
            )
        elif abs_pos==None:
            pos = rel_pos

        screw = ScrewBlock(screw_provider, counter_sunk_screw_provider).build(
            screw_size_category,
            block_thickness=8,
            taper=TaperOptions.Z_TAPER_ANGLE,
            taper_rotation=taper_rotation
        )
        screw_wp = screw["block"].translate([*pos, pos_error_margin])
        self.screws.append(screw_wp)
        return screw

    def add_corner_lid_screws(self, lid_screws_thickness_error_margin):
        # TODO support lid != Face.BOTTOM
        screw_size_category = "m3"
        screw_size = ScrewBlock().build(screw_size_category, 8)["size"]
        pw = self.size.inner_width
        pl = self.size.inner_length
        sw = screw_size[0]
        sl = screw_size[1]
        wt = self.size.wall_thickness
        corners = [
            ( (0+sw/2+wt,  0+sl/2+wt),    0 ),
            ( (0+sw/2+wt,  pl-sl/2-wt),  90 ),
            ( (pw-sw/2-wt, pl-sl/2-wt), 180 ),
            ( (pw-sw/2-wt, 0+sl/2+wt),  270 )
        ]
        for c in corners:
            screw_pos = c[0]
            screw_rotation = c[1]
            self.add_screw(
                screw_size_category=screw_size_category,
                abs_pos=screw_pos,
                pos_error_margin=lid_screws_thickness_error_margin,
                taper=TaperOptions.Z_TAPER_ANGLE,
                taper_rotation=screw_rotation
            )

    def assemble(self, walls_explosion_factor=1.0, lid_panel_shift=0.0):
        for panel in self.panels.values():
            panel.assemble()

        panels_assembly, panels_masks_assembly = self._build_panels_assembly(walls_explosion_factor, lid_panel_shift)
        frame_assembly = self._build_frame_assembly(panels_masks_assembly)
        lid_screws_assembly = self._build_lid_screws_assembly()

        footprints_assembly = self._build_debug_assembly([("footprint_in", "I"), ("footprint_out", "O")], walls_explosion_factor, lid_panel_shift)
        holes_assembly = self._build_debug_assembly([("hole", "")], walls_explosion_factor, lid_panel_shift)
        other_debug_assembly = self._build_debug_assembly([("other", "")], walls_explosion_factor, lid_panel_shift)

        self.debug = (
            cq.Assembly(None, name="Box")
                .add(footprints_assembly, name="Footprints")
                .add(holes_assembly, name="Holes")
                .add(other_debug_assembly, name="Others")
                .add(panels_masks_assembly, name="Panels masks")
        )
        self.assembly = (
            cq.Assembly(None, name="Box")
                .add(panels_assembly, name="Panels")
                .add(frame_assembly, name="Frame")
                .add(lid_screws_assembly, name="Lid screws", color=cq.Color(0.6, 0.45, 0.8))
                .add(self.debug, name="Debug")
        )
        self.printable_assembly = (
            cq.Assembly(None, name="Box")
                .add(panels_assembly, name="Panels")
                .add(frame_assembly, name="Frame")
                .add(lid_screws_assembly, name="Lid screws", color=cq.Color(0.6, 0.45, 0.8))
        )
        return self

    def _get_debug(self, panel: Panel, assembly_name="combined"):
        if assembly_name in panel.debug_assemblies:
            return panel.debug_assemblies[assembly_name]
        return None

    def _build_panels_assembly(self, walls_explosion_factor, lid_panel_shift):
        a = cq.Assembly(None)
        masks_a = cq.Assembly(None)
        for face, size, position, alpha in self.panels_specs:
            panel: Panel = self.panels[face]
            translated_mask = panel.mask.translate(position)
            masks_a.add(translated_mask, name=(face.label + " mask"), color=cq.Color(0, 1, 0))
            if face == Face.BOTTOM:
                position = (position[0], position[1], position[2] - lid_panel_shift)
            #if face == Face.BOTTOM:
            #    continue
            translated_panel = panel.panel.translate(explode(position, walls_explosion_factor))
            a.add(translated_panel, name=face.label)
        return (a, masks_a)

    def _build_frame_assembly(self, panels_masks_assembly) -> cq.Workplane:
        """
        Get the frame of the enclosure (the masks of the panels will be .cut during the assembly).
        In this implemenmtation, it's only possible to remove the fillet
        on the top and bottom of the enclosure (TODO: better implementation needed when lid_on_faces != [Face.BOTTOM] allowed),
        but you can to override the method if you need something more custom (and feel free to contribute ;) )
        """
        # TODO: when implementing the lid on other faces than BOTTOM, will have to remake that code better

        wall_thickness = self.size.wall_thickness

        shell_faces_filter = []
        shell_size = [
            self.size.inner_width - wall_thickness*2,
            self.size.inner_length - wall_thickness*2,
            self.size.inner_thickness - wall_thickness*2
        ]
        shell_translate_z = 0

        if self.no_fillet_top:
            shell_faces_filter.append("+Z")
            shell_size[2] = shell_size[2] + wall_thickness
            shell_translate_z = shell_translate_z# + wall_thickness
        if self.no_fillet_bottom:
            shell_faces_filter.append("-Z")
            shell_size[2] = shell_size[2] + wall_thickness
            shell_translate_z = shell_translate_z - wall_thickness

        no_faces_filter = "+Z and -Z"  # as two faces cannot be true at once--cheeky way to avoid `.faces("")`
        shell_faces = no_faces_filter if len(shell_faces_filter) == 0 else " or ".join(shell_faces_filter)

        shell = (
            cq.Workplane()
                .box(*shell_size, centered=(True, True, False))
                .faces(shell_faces)
                .shell(wall_thickness)
                .translate([0, 0, shell_translate_z])
        )

        return shell.cut(panels_masks_assembly.toCompound())

    def _build_lid_screws_assembly(self):
        a = cq.Assembly(None)
        for idx, s in enumerate(self.screws):
            a.add(s, name=f"Screw #{idx}", color=cq.Color(0.6, 0.45, 0.8))
        return a

    def _cut_panels_masks_from_frame(self, frame: cq.Workplane) -> cq.Workplane:
        for face, size, position, alpha in self.panels_specs:
            panel: Panel = self.panels[face]
            translated_mask = panel.mask.translate(position)
            frame = frame.cut(translated_mask)
        return frame

    def _build_debug_assembly(self, assemblies_specs, walls_explosion_factor, lid_panel_shift):
        a = cq.Assembly(None)
        for face, size, position, alpha in self.panels_specs:
            if face == Face.BOTTOM:
                position = (position[0], position[1], position[2] - lid_panel_shift)
            panel: Panel = self.panels[face]
            for assembly_type, assembly_name_suffix in assemblies_specs:
                debug_assembly = self._get_debug(panel, assembly_type)
                if debug_assembly != None:
                    translated_debug = debug_assembly.translate(explode(position, walls_explosion_factor))
                    a.add(translated_debug, name=(f"{face.label} {assembly_name_suffix}"))
        return a

In [7]:
class AssemblyPart:
    def __init__(self, workplane: cq.Workplane, name: str, color: cq.Color):
        self.workplane: cq.Workplane = workplane
        self.name: str = name
        self.color: cq.Color = color

    def as_assembly_add_parameters(self):
        return (self.workplane, None, self.name, self.color)

class DebugObjects:
    class Footprint:
        def __init__(self):
            self.inside: cq.Workplane = None
            self.outside: cq.Workplane = None

    def __init__(self):
        # Space taken by the component (anything: PCB, bolts, caps, etc.).
        # Used for visualisation only.
        self.footprint = DebugObjects.Footprint()

        # Materialisation of the actual hole in the enclosure (e.g. a ~9.35mm extruded circle hole for a female jack).
        # Used for visualisation only.
        self.hole = None

        self.others = {}

class PartSize:
    def __init__(self):
        self.width: float = 0
        self.length: float = 0
        self.thickness: float = 0

class Part:
    def __init__(self):
        self.part: cq.Workplane = None
        self.mask: cq.Workplane = None
        self.assembly_parts: List[AssemblyPart] = None

        self.size = PartSize()
        self.inside_footprint = None  # used by the layout builder  # TODO
        self.outside_footprint = None  # used by the layout builder
        self.inside_footprint_offset = None  # how far is it from 0,0
        self.outside_footprint_offset = (0, 0)  # currently, it is assumed the center of the outside-facing side of the part is at 0,0

        self.debug_objects = DebugObjects()

    def assembly_parts_to_cq_assembly(self):
        if self.assembly_parts is None:
            return None
        panel_assembly = cq.Assembly()
        for part in self.assembly_parts:
            panel_assembly.add(part.workplane, name=part.name, color=part.color)
        return panel_assembly

    def show(self):  # TODO delete
        print("debug show() part " + self.__class__.__name__ + " - TODO delete")

    def validate(self):
        print("VALIDATING CLASS: " + self.__class__.__name__)

In [63]:
from enum import Enum

class FanSize(Enum):
    _40_MM = 1
    _30_MM = 2
    _25_MM = 3

bp_size = 45
enclosure_wall_thickness = 1.2

base_plate = (
    cq.Workplane("front")
        .box(bp_size, bp_size, enclosure_wall_thickness, centered=(True, True, False))
)

show(base_plate)

@staticmethod
def _get_fan_screws_blocks(
    fan_size: FanSize,
    block_thickness: float,
    screw_block_taper_option: TaperOptions = TaperOptions.XY_TAPER,
    screw_block_taper_rotation: float = 0,
    screw_block_taper_on: str = "1111"  # should taper that specific screw block? same order as screws_pos
):
    screw = ScrewBlock().build("m2", block_thickness, screw_hole_depth=block_thickness-1.2, taper=screw_block_taper_option, taper_rotation=screw_block_taper_rotation)
    screw_no_taper = ScrewBlock().build("m2", block_thickness, screw_hole_depth=block_thickness-1.2)

    distance_between_screws = None  # center to center
    if fan_size == FanSize._25_MM: distance_between_screws = 16.9 + 2.9
    elif fan_size == FanSize._30_MM: distance_between_screws = 20.8 + 3.2
    elif fan_size == FanSize._40_MM: distance_between_screws = 28.9 + 3.2
    else: raise ValueError("Unknown FanSize")

    hdbs = distance_between_screws/2

    screws_pos = [
        (-hdbs,  hdbs),  # TR
        ( hdbs,  hdbs),  # TL
        ( hdbs, -hdbs),  # BL
        (-hdbs, -hdbs),  # BR
    ]

    screws = cq.Assembly()
    masks = cq.Assembly()
    for idx, sp in enumerate(screws_pos):
        current_screw = screw if screw_block_taper_on[idx] == "1" else screw_no_taper  # see apologies in constructor :D
        screws.add(current_screw["block"].translate([*sp, 0]))
        masks.add(current_screw["mask"].translate([*sp, 0]))

    return {
        "screws": screws.toCompound(),
        "masks": masks.toCompound(),
        #"base_plate": base_plate.translate([0, 0, -enclosure_wall_thickness])
    }

fan_screws = get_fan_screws_blocks(FanSize._25_MM, block_thickness=5)

a = (
    cq.Assembly()
        .add(fan_screws["base_plate"], name="base")
        .add(fan_screws["screws"], name="Screws")
        .add(fan_screws["masks"], name="Masks")
)

show(a)

from cadquery import exporters
exporters.export(a.toCompound(), 'models/fan-40mm-v1.stl')

100% ⋮————————————————————————————————————————————————————————————⋮ (3/3)  0.41s


In [86]:
import math
from enum import Enum
from typing import Union

class FanSize(Enum):
    _40_MM = 1
    _30_MM = 2
    _25_MM = 3

class RectAirVentPart(Part):
    """
    Class to make basic rectangular vents, with optional screws for fan.

    Needs work. Hasn't been tested much besides the default parameters.
    """
    def __init__(
        self,
        enclosure_wall_thickness,
        width = 30,
        length = 18,
        thickness = 5,
        margin = 1,
        hole_angle = 25,
        hole_width = 1.6,
        distance_between_holes = 3,
        taper = True,
        taper_margin = 2,
        taper_angle = 35,
        with_fan_screws: Union[FanSize, None] = FanSize._25_MM,
        fan_screws_taper_mode: TaperOptions = TaperOptions.XY_TAPER,
        fan_screws_taper_rotation: float = 0,
        fan_screws_taper_on: str = "1100"  # TODO cleanup one day; sorry future self, just want to be done with it :shrug:
    ):
        super().__init__()

        # Board
        board = (
            cq.Workplane("front")
                .box(width, length, thickness, centered=(True, True, False))
        )

        # Add holes
        single_hole = (
            cq.Workplane("front")
                .box(hole_width, length, thickness*6, centered=(True, True, False))
                .translate([0, 0, -(thickness*3)])
                .rotate((0, 0, 0), (0, 1, 0), hole_angle)
        )

        all_holes = cq.Assembly()
        hole_x = 0
        while hole_x < width/2:
            all_holes.add(single_hole.translate([hole_x, 0, 0]))
            all_holes.add(single_hole.translate([-hole_x, 0, 0]))
            hole_x = hole_x + distance_between_holes

        board = board.cut(all_holes.toCompound())

        # Add frame
        board = (
            cq.Workplane("front")
                .box(width + margin*2, length + margin*2, thickness, centered=(True, True, False))
                .cut(cq.Workplane("front").box(width, length, thickness, centered=(True, True, False)))
                .add(board)
        )        

        # Pyramid
        if taper:
            pyramid = (
                cq.Workplane("front")
                    .rect(width + margin*2 + taper_margin*2, length + margin*2 + taper_margin*2)
                    .extrude(30, taper=taper_angle)
                    .cut(cq.Workplane("front").rect(width + 4, length + 4).extrude(40).translate([0, 0, thickness - enclosure_wall_thickness]))
                    .cut(cq.Workplane("front").rect(width, length).extrude(thickness - enclosure_wall_thickness).translate([0, 0, 0]))
                    .translate([0, 0, enclosure_wall_thickness])
            )
            board = board.add(pyramid)

        # Fan screws
        if with_fan_screws is not None:
            fan_screws_a = RectAirVentPart._get_fan_screws_blocks(
                with_fan_screws,
                block_thickness=thickness-1,
                screw_block_taper_option=fan_screws_taper_mode,
                screw_block_taper_rotation=fan_screws_taper_rotation,
                screw_block_taper_on=fan_screws_taper_on
            )
            fan_screws = fan_screws_a["screws"].translate([0, 0, 1])
            fan_masks = fan_screws_a["masks"].translate([0, 0, 1])

            board = board.cut(fan_masks).add(fan_screws)

        # Mask & footprint
        mask = (
            cq.Workplane("front")
                .box(width, length, enclosure_wall_thickness, centered=(True, True, False))
        )

        self.part = board
        self.mask = mask

        self.size.width     = width
        self.size.length    = length
        self.size.thickness = thickness

        self.inside_footprint = (self.size.width, self.size.length)
        self.inside_footprint_offset = (0, 0)
        self.outside_footprint = (self.size.width, self.size.length)
        inside_footprint_thickness = 4
        footprint_in = (
            cq.Workplane("front")
                .rect(self.size.width, self.size.length)
                .extrude(inside_footprint_thickness)
                .translate([0, 0, enclosure_wall_thickness])
        )
        outside_footprint_thickness = 2
        footprint_out = (
            cq.Workplane("front")
                .rect(*self.outside_footprint)
                .extrude(outside_footprint_thickness)
                .translate([-0, 0, -outside_footprint_thickness])
        )
        self.debug_objects.footprint.inside  = footprint_in
        self.debug_objects.footprint.outside = footprint_out
        
        self.debug_objects.hole = all_holes.toCompound()

    @staticmethod
    def _get_fan_screws_blocks(
        fan_size: FanSize,
        block_thickness: float,
        screw_block_taper_option: TaperOptions = TaperOptions.XY_TAPER,
        screw_block_taper_rotation: float = 0,
        screw_block_taper_on: str = "1111"  # should taper that specific screw block? same order as screws_pos
    ):
        screw = ScrewBlock().build("m2", block_thickness, screw_hole_depth=block_thickness-1.2, taper=screw_block_taper_option, taper_rotation=screw_block_taper_rotation)
        screw_no_taper = ScrewBlock().build("m2", block_thickness, screw_hole_depth=block_thickness-1.2)

        distance_between_screws = None  # center to center
        if fan_size == FanSize._25_MM: distance_between_screws = 16.9 + 2.9
        elif fan_size == FanSize._30_MM: distance_between_screws = 20.8 + 3.2
        elif fan_size == FanSize._40_MM: distance_between_screws = 28.9 + 3.2
        else: raise ValueError("Unknown FanSize")

        hdbs = distance_between_screws/2

        screws_pos = [
            (-hdbs,  hdbs),  # TR
            ( hdbs,  hdbs),  # TL
            ( hdbs, -hdbs),  # BL
            (-hdbs, -hdbs),  # BR
        ]

        screws = cq.Assembly()
        masks = cq.Assembly()
        for idx, sp in enumerate(screws_pos):
            current_screw = screw if screw_block_taper_on[idx] == "1" else screw_no_taper  # see apologies in constructor :D
            screws.add(current_screw["block"].translate([*sp, 0]))
            masks.add(current_screw["mask"].translate([*sp, 0]))

        return {
            "screws": screws.toCompound(),
            "masks": masks.toCompound(),
            #"base_plate": base_plate.translate([0, 0, -enclosure_wall_thickness])
        }

In [87]:
        # hole_angle=25,
        # hole_width=1.6,
        # distance_between_holes=3,

# vent = AirVentPart(2, width=30, length=25, thickness=5, with_fan_screws=FanSize._30_MM, hole_angle=25, hole_width=1.6, distance_between_holes=3)
vent = AirVentPart(2, width=25, length=20, thickness=5, with_fan_screws=FanSize._25_MM, hole_angle=25, hole_width=1.6, distance_between_holes=3)
# vent = AirVentPart(2, width=40, length=20, thickness=5, with_fan_screws=FanSize._25_MM, hole_angle=25, hole_width=1, distance_between_holes=1.5)
# vent = AirVentPart(2, width=25, length=25, thickness=4, hole_angle=25, hole_width=1, distance_between_holes=1.5)
# vent = AirVentPart(2, width=30, length=26, thickness=4, hole_angle=40, hole_width=2, distance_between_holes=4)
# vent = AirVentPart(2, width=30, length=26, thickness=8, taper_margin=4, hole_angle=15, hole_width=1.2, distance_between_holes=2.1)
# vent = AirVentPart(2, width=30, length=26, thickness=3, hole_angle=35, hole_width=0.6, distance_between_holes=1.35)

p = Panel(Face.LEFT, 90, 36, 2)
p.add("Vent", vent, rel_pos=(0, 0))
p.assemble()
show(p.panel)
# show_part(vent)
# show(p.debug_assemblies["combined"])

LEFT: adding part 'Vent'
100% ⋮————————————————————————————————————————————————————————————⋮ (3/3)  0.41s


In [ ]:
        self,
        enclosure_wall_thickness,
        width=30,
        length=18,
        thickness=4,
        margin=1,
        hole_angle=25,
        hole_width=1.6,
        distance_between_holes=3,
        taper=True,
        taper_margin=2,
        taper_angle=45

In [130]:
class PyramidSupportPart(Part):
    def __init__(self, enclosure_wall_thickness, support_height):
        super().__init__()

        base_size = 8
        top_size = 5

        # Board
        board = (
            cq.Workplane("front")
                .rect(base_size, base_size)
                .extrude(support_height, taper=20)
        )
        board = board.add(
            cq.Workplane("front")
                .rect(top_size, top_size)
                .extrude(support_height)
        )

        self.part = board
        self.mask = (
            cq.Workplane("front")
                .box(base_size, base_size, enclosure_wall_thickness, centered=(True, True, False))
        )

        self.size.width     = base_size
        self.size.length    = base_size
        self.size.thickness = support_height

        self.inside_footprint = (self.size.width, self.size.length)
        self.inside_footprint_offset = (0, 0)
        self.outside_footprint = (0, 0)
        self.debug_objects.footprint.inside  = board
        self.debug_objects.footprint.outside = None
        self.debug_objects.hole = None

In [131]:
support = PyramidSupportPart(2, 17)

show_part(support)

100% ⋮————————————————————————————————————————————————————————————⋮ (5/5)  0.02s


In [132]:
from cq_enclosure_builder import PartFactory as pf

vent_l = AirVentPart(2, width=30, length=25, thickness=5, with_fan_screws=FanSize._30_MM, hole_angle=25, hole_width=1.6, distance_between_holes=3)
vent_r = AirVentPart(2, width=30, length=25, thickness=5, hole_angle=25, hole_width=1.6, distance_between_holes=3)

vent_f1 = AirVentPart(2, width=40, length=20, thickness=5, with_fan_screws=FanSize._25_MM, hole_angle=25, hole_width=1, distance_between_holes=1.5)
vent_b1 = AirVentPart(2, width=30, length=26, thickness=8, with_fan_screws=FanSize._25_MM, taper_margin=4, hole_angle=15, hole_width=1.2, distance_between_holes=2.1)

vent_f2 = AirVentPart(2, width=30, length=26, thickness=4, hole_angle=40, hole_width=2, distance_between_holes=4)
vent_b2 = AirVentPart(2, width=30, length=26, thickness=3, hole_angle=35, hole_width=0.6, distance_between_holes=1.35)

enclosure = Enclosure(EnclosureSize(100, 60, 42, 2))
enclosure.add_part_to_face(Face.LEFT, "Vent L", vent_l, rel_pos=(0, 0))
enclosure.add_part_to_face(Face.RIGHT, "Vent R", vent_r, rel_pos=(0, 0))
enclosure.add_part_to_face(Face.FRONT, "Vent F1", vent_f1, rel_pos=(-20, 0))
enclosure.add_part_to_face(Face.FRONT, "Vent F2", vent_f2, rel_pos=(20, 0))
enclosure.add_part_to_face(Face.BACK, "Vent B1", vent_b1, rel_pos=(-20, 0))
enclosure.add_part_to_face(Face.BACK, "Vent B2", vent_b2, rel_pos=(20, 0))
enclosure.add_part_to_face(Face.TOP, "SPST 1", pf.build_button(), abs_pos=(20, 30))
enclosure.add_part_to_face(Face.BOTTOM, "Support SPST", support, abs_pos=(20, 30))
enclosure.assemble()

show(enclosure.assembly, hide_contains=["Masks", "FRONT"])


# from cadquery im/zzzzz

LEFT: adding part 'Vent L'
RIGHT: adding part 'Vent R'
FRONT: adding part 'Vent F1'
FRONT: adding part 'Vent F2'
BACK: adding part 'Vent B1'
BACK: adding part 'Vent B2'
VALIDATING CLASS: ButtonSpstPbs24b4Part
TOP: adding part 'SPST 1'
BOTTOM: adding part 'Support SPST'
100% ⋮————————————————————————————————————————————————————————————⋮ (68/68)  4.57s


100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.25s
100% ⋮————————————————————————————————————————————————————————————⋮ (3/3)  0.42s


In [452]:
screw = ScrewBlock().build("m2", 3, screw_hole_depth=1)
show(screw["block"])

In [363]:
# ENCLOSURE_INNER_WIDTH =     220
# ENCLOSURE_INNER_LENGTH =    150
# ENCLOSURE_INNER_THICKNESS = 38
# ENCLOSURE_WALLS_THICKNESS = 2

# enclosure_size = EnclosureSize(ENCLOSURE_INNER_WIDTH, ENCLOSURE_INNER_LENGTH, ENCLOSURE_INNER_THICKNESS, ENCLOSURE_WALLS_THICKNESS)
# enclosure = Enclosure(size=enclosure_size, no_fillet_bottom=True, no_fillet_top=True)
# enclosure.add_part_to_face(Face.TOP, "Screen", pf.build_screen(), rel_pos=(0, -4))
# enclosure.add_part_to_face(Face.TOP, "SPST 1", pf.build_button(), abs_pos=(20, 30))
# enclosure.add_part_to_face(Face.TOP, "SPST 2", pf.build_button(), abs_pos=(20, 100))
# enclosure.add_part_to_face(Face.TOP, "SPST 3", pf.build_button(), abs_pos=(200, 30))
# enclosure.add_part_to_face(Face.TOP, "Encoder", pf.build_encoder(), abs_pos=(200, 100))
# for i in range(0, 4):
#     enclosure.add_part_to_face(Face.BACK, f"Js{i}", pf.build_jack(part_type='3.5mm XXX'), abs_pos=(8, 4.5 + i*9))
# for i in range(0, 7):
#     enclosure.add_part_to_face(Face.BACK, f"Jb{i}", pf.build_jack(), abs_pos=(24 + i*18, 10))
#     enclosure.add_part_to_face(Face.BACK, f"Jt{i}", pf.build_jack(), abs_pos=(24 + i*18, 18+9))
# enclosure.add_part_to_face(Face.LEFT, "USB", pf.build_usb_a(), rel_pos=(0, 0))
# enclosure.add_part_to_face(Face.LEFT, "USB V", pf.build_usb_a(orientation_vertical=True), rel_pos=(40, 0))
# enclosure.add_part_to_face(Face.RIGHT, "USB C", pf.build_usb_c(), rel_pos=(0, 0))
# enclosure.add_part_to_face(Face.RIGHT, "USB C", pf.build_usb_c(), rel_pos=(0, 0))

# enclosure.assemble(walls_explosion_factor=1.0, lid_panel_shift=100)

# show(enclosure.printable_assembly, hide_contains=["gfg"])

VALIDATING CLASS: Hdmi5InchJrp5015Part
TOP: adding part 'Screen'
VALIDATING CLASS: ButtonSpstPbs24b4Part
TOP: adding part 'SPST 1'
VALIDATING CLASS: ButtonSpstPbs24b4Part
TOP: adding part 'SPST 2'
VALIDATING CLASS: ButtonSpstPbs24b4Part
TOP: adding part 'SPST 3'
VALIDATING CLASS: EncoderEc11Part
TOP: adding part 'Encoder'
VALIDATING CLASS: Jack3_5mmXxxPart
BACK: adding part 'Js0'
VALIDATING CLASS: Jack3_5mmXxxPart
BACK: adding part 'Js1'
VALIDATING CLASS: Jack3_5mmXxxPart
BACK: adding part 'Js2'
VALIDATING CLASS: Jack3_5mmXxxPart
BACK: adding part 'Js3'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jb0'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jt0'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jb1'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jt1'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jb2'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jt2'
VALIDATING CLASS: Jack6_35mmPj612aPart
BACK: adding part 'Jb3'

ValueError: Unique name is required

In [484]:
ss = ScrewBlockX().m2(5, taper=TaperOptions.XY_TAPER, taper_rotation=0)

show(ss["block"])

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.33s
